# Project 2

Morgan Elder<br>
Ben Vuong<br>
CS 4320

1. Write a brief (no more than a paragraph) note on the importance of the choice of 
parameter settings in evolutionary algorithms. Or argue, with well-articulated, precise 
reasons (one paragraph only), that parameters do not really matter in EC.  

I would say that when working with evolutionary algorithms, diversity is very important for the growth and success of the evolutionary algorithms. There should not be one indiviudual within the population that over takes the other. If that would happen then there would be stagnation within the growth of the algorithms. In order to prevent such an event of happening and encourage growth and diversity, it is important to focus on the choice of parameters settings in evolutionary algorithms. If good parameters were choosen, there will be higher chances or growth and diversity to happen and the inverse would result in a higher chance of stangnation.

Evolutionary algorithm (EA) parameters directly influence the exploration with diverse populations and exploitation of fit individuals. Parameters include all of the algorithm design choices such as genetic operators, operator parameters, stopping conditions, population size and seed. For example, parameters like the mutation rate or cross over probability can increase exploration whereas selection/sampling method can increase exploitation. Because EAs are stochastic, the applicability of an EA, more specfically the EA parameters, accross different problems, problem instances, and sizes must be compared using statistical methods and many runs that are *iid*. Therefore, parameter settings are an important part of the optimization process that must strike a balance between generalizability, computational cost, and performance.

# De Jong Function 5: Variation of Shekel's Function 

The objective of this project is to minimize Shekel's function using parameters from De Jong function #5:

$$f(\vec{x}) = \left(0.002 + \sum_{i=1}^{25}\frac{1}{i + (x_1 - a_{1i})^6 + (x_2 - a_{2i})^6}\right)^{-1}$$

where $$
\textbf{a} =
 \begin{pmatrix}
    -32 & -16 & 0 & 16 & 32 & -32 & \ldots & 0 & 16 & 32 \\
    -32 & -32 & -32 & -32 & -32 & -16 & \ldots & 32 & 32 & 32
 \end{pmatrix}$$

The number of dimensions is 2 and the input domain is $-65.536 \le x_i \le 65.536$ for $i=1,2$. 










In [1]:
#import modules
import numpy as np
from enum import Enum
import warnings
import inspect
from typing import Union

## Optimization Goals

Use the Optimization_Goal class as an enum type in order to define a goal variable as either min or max.

In [2]:
class Optimization_Goal(Enum):
  MINIMIZE = 1
  MAXIMIZE = 2

## Optimization Problems

Genetic algorithms are suited for optimization problems. The Problem class is used to identify subclasses as optimization problems.

In [3]:
class Problem():
  """The Problem class defines the problems contains the definitions of
  of problems. Problems include any information such as the optimization
  goal (min/max), objective function, input dimension, domain/range constraints, 
  and more."""
  pass
  
class De_Jong_Function_5(Problem):
  """De Jong function #5 is variation of Shekel's foxeholes problem involving
  25 local minima, 2 dimensions, and predefined constants.
  This problem is suited for multimodal optimization."""
  dimensions=2
  goal=Optimization_Goal.MINIMIZE
  upper_bounds = [65.536, 65.536]
  lower_bounds = [-65.536, -65.536]
  constants_array = np.array(
      [[-32., -16., 0., 16., 32., -32., -16., 0., 16., 32., -32., -16., 0., 
        16., 32., -32., -16., 0., 16., 32., -32., -16., 0., 16., 32.],
        [-32., -32., -32., -32., -32., -16., -16., -16., -16., -16., 0., 0., 
        0., 0., 0., 16., 16., 16., 16., 16., 32., 32., 32., 32., 32.]]
    )
  def objective_function(self, X):
    # array representing the 1st to 25th elements of the summation
    array_i = np.arange(1, self.constants_array.shape[1]+1, step=1)
    results = (0.002 + np.sum(1/(array_i 
                        + (X[:,[0]] - self.constants_array[0])**6
                        + (X[:, [1]] - self.constants_array[1])**6), axis=1))**-1
    return results    

## Genetic Operators

In [4]:
class Genetic_Operator:
    """The Genetic_Operator base class defines the genetic operators used in
    genetic algorithms. These include crossover and mutation."""
    pass

### Selection

In [5]:
class Selection(Genetic_Operator):
    """The Selection class defines the selection operator used in genetic
    algorithms. This includes the selection of parents and the selection of
    survivors."""
    
    def __init__(self, selection_type: str="proportional", 
                 truncation_percentage: float=0.2, expected_best_copies: float=1.2):
      # dictionary of selection types and their corresponding methods
      selection_types = { "proportional": self.proportional,
                          "deterministic_tournament": self.deterministic_tournament,
                          "linear_ranking": self.linear_ranking,
                          "truncation": self.truncation }
      # check if selection type is valid
      if selection_type not in selection_types:
        raise ValueError(f"Invalid selection type: {selection_type}")
      self.selection_type = selection_types[selection_type]

      if selection_type == "truncation":
        if truncation_percentage < 0 or truncation_percentage > 1:
          raise ValueError(f"Invalid truncation percentage: {truncation_percentage}")
        self.truncation_percentage = truncation_percentage

      if selection_type == "linear_ranking":
        # expected_best_copies must be 1 < x <= 2
        if expected_best_copies <= 1 or expected_best_copies > 2:
          raise ValueError(f"Invalid expected copies of best: {expected_best_copies}")
        self.expected_best_copies = expected_best_copies
    
    def get_operator_args(self):
      """The get_operator_args method gets the arguments of the selection
      method."""
      return inspect.getfullargspec(self.selection_type).args[1:]
    
    def proportional(self, population, fitness_values, goal, rng):
      if (goal is Optimization_Goal.MAXIMIZE):
        weights = fitness_values / np.sum(fitness_values, axis=0)
      else:
        fitness_values += 0.0001
        weights = (1/fitness_values)/np.sum(1/fitness_values, axis=0)
      
      indexes = np.arange(population.shape[0])
      indexes= rng.choice(indexes, size=population.shape[0], p=weights, replace=True)

      return population[indexes, :]
    
    def deterministic_tournament(self, population):
      # TODO implement deterministic tournament selection
      #Psuedo Code/ general idea:
      #take a random sample with replacement from the population, sample K individuals where K is how many opponents will enter the trounament
      #take the max out of the K individuals that was sampled
      #repeat N times where N = popsize
      #for n in range(popsize)
      # tournament = random.sample(population, K) 
      # winner = max(tournament, key=lambda individual: fitness_value[population.index(individual)]))
      return population
    
    def linear_ranking(self,fitness_values, population, goal, rng):
      # TODO implement linear ranking selection
      #Psuedo Code/ general idea:
      #sort the individuals by rank, then find the probability from rank
      #rank = np.argsort(np.argsort(-fitness_values))# find the rank of each individual
      #prob = (2-(2*rank)/(len(population)-1)) / len(population)
      #selectedIndexs = rng.choice(indexes, size=population.shape[0], p=prob= replace=False)
      #population = population[selectedIndexs,:]

      if goal == Optimization_Goal.MAXIMIZE:
        # sort the fitness values from smallest to largest
        sorted_indexes = np.argsort(fitness_values)
      elif goal == Optimization_Goal.MINIMIZE:
        # sort the fitness values from largest to smallest
        sorted_indexes = np.argsort(fitness_values)[::-1]

      rank = np.arange(1, population.shape[0]+1)
      probabilities = self.calc_rank_based_probabilities(rank, population.shape[0])

      selected_indexes = rng.choice(sorted_indexes, size=population.shape[0],
                                    p=probabilities, replace=True)

      return population[selected_indexes, :]
    
    def calc_rank_based_probabilities(self, rank, population_size):
      expected_worst_copies = 2 - self.expected_best_copies
      probabilities = expected_worst_copies \
        + ((rank-1)/(population_size-1))*(self.expected_best_copies - expected_worst_copies)
      probabilities /= population_size
      return probabilities
      
    def truncation(self, population, goal, fitness_values, rng):
      # TODO implement truncation selection
      #Psuedo Code/ general idea:
      #Take the individuals and their corresponding fittness and sort them from least to best
      #Then take the top K best indivudals and their fitness 
      #sortedIndex=np.argsort(fitness_values)[::-1]
      #selectedIndexs = sortedIndex[:K]
      #population = population[selectedIndexs,:]

      truncation_size = int(self.truncation_percentage * population.shape[0])
      remaining_size = population.shape[0] - truncation_size
      
      if goal == Optimization_Goal.MAXIMIZE:
        top_indexes = np.argpartition(fitness_values, -truncation_size, axis=0)[-truncation_size:]
      elif goal == Optimization_Goal.MINIMIZE:
        top_indexes = np.argpartition(fitness_values, truncation_size, axis=0)[:truncation_size]


      fill_bottom_indexes = rng.choice(top_indexes, remaining_size, replace=True)
      return_indexes = np.concatenate((top_indexes, fill_bottom_indexes))

      return population[return_indexes, :]

    def run(self, **kwargs):
      """The run method runs the selection operator."""
      return self.selection_type(**kwargs)
        

SyntaxError: invalid syntax (736940007.py, line 44)

### Cross Over

In [ ]:
class Crossover(Genetic_Operator):
    """The Crossover class defines the crossover operator used in genetic
    algorithms."""
    
    def __init__(self, crossover_type: str="single_point", rate: float=0.8):
      crossover_types = { "single_point": self.single_point }
      if crossover_type not in crossover_types:
        raise ValueError(f"Invalid crossover type: {crossover_type}")
      self.crossover_type = crossover_types[crossover_type]
      self.rate = rate
      

    def single_point(self, population, rng):
      offspring = np.zeros_like(population)
      groups = rng.integers(0, population.shape[0], size=(population.shape[0],2))
      is_group_selected = rng.uniform(0, 1, size=population.shape[0]) <= self.rate
      for i, group in enumerate(groups):
        if (is_group_selected[i]):
          cross_point = self.generate_k_cross_points(population[group], rng)
          offspring[i] = self.swap_at_cross_point(population[group, :], cross_point)
        else:
          offspring[i] = population[group[0],:]
               
      return offspring
    
    def generate_k_cross_points(self, group, rng, k=1):
      """Generate the location of the cross points."""
      return np.sort(rng.choice(range(1, group.shape[1]), size=k, replace=False))[0]
    
    def swap_at_cross_point(self, group, cross_point):
      """Return the vector with the elements swapped at the cross point."""
      group[0,cross_point:] = group[1,cross_point:]
      return group[0]

    def run(self, **kwargs):
      """The run method runs the crossover operator."""
      return self.crossover_type(**kwargs)

    def get_operator_args(self):
      """The get_operator_args method gets the arguments of the crossover
      method."""
      return inspect.getfullargspec(self.crossover_type).args[1:]


### Mutation

In [ ]:
class Mutation(Genetic_Operator):
    """The Mutation class defines the mutation operator used in genetic
    algorithms."""
    
    def __init__(self, mutation_type: str="percent_of_range", rate: float=0.1):
      mutation_types = { "percent_of_range": self.percent_of_range }
      if mutation_type not in mutation_types:
        raise ValueError(f"Invalid mutation type: {mutation_type}")
      self.mutation_type = mutation_types[mutation_type]
      self.rate = rate

    def percent_of_range(self, population, rng, lower_bounds, upper_bounds):
      """plus or minus a small percent of interval"""

      upper_bounds = np.array(upper_bounds)
      lower_bounds = np.array(lower_bounds)
      
      mutation_amount = abs(upper_bounds - lower_bounds) * 0.02
      isMutating = rng.uniform(0,1,population.shape) <= self.rate
      mutation_amount = mutation_amount * rng.choice([-1,1], size=population.shape) * isMutating
      
      return population + mutation_amount

    def run(self, **kwargs):
      """The run method runs the mutation operator."""
      return self.mutation_type(**kwargs)

    def get_operator_args(self):
      """The get_operator_args method gets the arguments of the mutation
      method."""
      return inspect.getfullargspec(self.mutation_type).args[1:]

## Stopping Conditions

In [ ]:
class Stopping_Condition:
    """The Stopping_Condition base class defines the stopping conditions used
    in genetic algorithms. These include the number of generations and the
    fitness threshold."""

    
    pass

class Max_Generations(Stopping_Condition):
    """The number_of_generations class defines the stopping condition based on
    the number of generations."""
    def __init__(self, max_generations: int=100):
      self.max_generations = max_generations
    
    def is_running(self, generation: int):
        """The is_running method checks if the algorithm should continue running
        based on the number of generations."""
        running = True
        if generation >= self.max_generations:
            running = False
        return running  
    
    def increase_max_generations(self, increase: int=100):
        """The increase_max_generations method increases the maximum number of
        generations by a specified amount."""
        self.max_generations += increase

    

## Genetic Algorithm

In [ ]:
class Genetic_Algorithm():
    problem : Problem
    algorithm_steps : list[Genetic_Operator] = []
    stopping_condition : Stopping_Condition
    generation : int = 0
    population : np.ndarray
    population_size : int
    fitness_values : np.ndarray
    
    def __init__(self, problem : Problem = None, algorithm_steps : list[Genetic_Operator] = [],
                 stopping_condition : Stopping_Condition = Max_Generations(),
                 seed : int = 1234, rng : np.random.Generator = None,
                 population_size : int = 100, population : np.ndarray = np.array([]), 
                  ):
        # loop locals and validate types
        for key, value in locals().items():
            match key:
                case "problem":
                    # check if problem is an instance of Problem
                    if (not isinstance(value, Problem)):
                        raise TypeError(f"problem must be an instance of Problem. Received {type(value)}")
                    self.problem = value
                case "algorithm_steps":
                    # check if algorithm_steps is a list
                    if (not isinstance(value, list)):
                        raise TypeError(f"algorithm_steps must be a list. Received {type(value)}")
                    # check if algorithm_steps is a list of Genetic_Operator
                    if (not all(isinstance(item, Genetic_Operator) for item in value)):
                        raise TypeError(f"algorithm_steps must be a list of Genetic_Operator. Received {type(value)}")
                    self.algorithm_steps = value
                case "stopping_condition":
                    # check if stopping_condition is an instance of Stopping_Condition
                    if (not isinstance(value, Stopping_Condition)):
                        raise TypeError(f"stopping_condition must be an instance of Stopping_Condition. Received {type(value)}")
                    self.stopping_condition = value
                case "population_size":
                    # check if population_size is an int or zero
                    if (not isinstance(value, int)):
                        raise TypeError(f"stopping_condition must be an instance of int. Received {type(value)}")
                    self.population_size = value
                case "population":
                    if (not isinstance(value, np.ndarray)):
                        raise TypeError(f"population must be an instance of np.ndarray. Received {type(value)}")

                    if (len(value) == 0):
                        self.population = self.generate_dataset()
                    else:
                        self.population = value
                case "seed":
                    # check if seed is an int
                    if (not isinstance(value, int)):
                        raise TypeError(f"seed must be an instance of int. Received {type(value)}")
                    self.seed = value
                case "rng":
                    if (value is None):
                        self.rng = np.random.default_rng(self.seed)
                    # check if rng is an instance of np.random.Generator
                    elif (not isinstance(value, np.random.Generator)):
                        raise TypeError(f"rng must be an instance of np.random.Generator. Received {type(value)}")
                    else:
                        self.rng = value

    

    def generate_dataset(self):
        """Randomly generate a population as a numpy array of size=(population size, dimensions)
        with values from the half-closed interval [min_x, max_x) from a uniform distribution"""
        # check if dimensions exist and is int
        if (not hasattr(self.problem, "dimensions")):
            raise Exception("The problem object should have a dimensions attribute defining the size (# of cols) the input vector has")
        elif (not isinstance(self.problem.dimensions, int)):
            raise TypeError(f"The problem dimensions should be an int. Received {self.problem.dimensions}")
        
        args = {}

        args["size"] = (self.population_size, self.problem.dimensions)
        if (hasattr(self.problem, "lower_bounds")):
            args["low"] = self.problem.lower_bounds
        if (hasattr(self.problem, "upper_bounds")):
            args["high"] = self.problem.upper_bounds

        return self.rng.uniform(**args)
            

    def add_step(self, step : Genetic_Operator):
        """Add a step to the algorithm. The step will be appended to the
        end of the algorithm."""
        # check if step is a Genetic_Operator
        if (not isinstance(step, Genetic_Operator)):
            raise TypeError(f"step must be a Genetic_Operator. Received {type(step)}")
        self.algorithm_steps.append(step)

    def add_steps(self, steps : list[Genetic_Operator]):
        """Add multiple steps to the algorithm. The steps will be appended to the
        end of the algorithm."""
        # check if steps is a list
        if (not isinstance(steps, list)):
            raise TypeError(f"steps must be a list. Received {type(steps)}")
        # check if steps is a list of Genetic_Operator
        if (not all(isinstance(item, Genetic_Operator) for item in steps)):
            raise TypeError(f"steps must be a list of Genetic_Operator. Received {type(steps)}")
        self.algorithm_steps.extend(steps)

    def set_problem(self, problem : Problem):
        """Set the problem to be solved. If a problem has already been set,
        it will be overwritten and a warning will be raised."""
        if (self.problem is not None):
            # 
            warnings.warn(f"Problem already set to {self.problem}. Overwriting with {problem}.")
        self.problem = problem

    def run(self):
        """Run the genetic algorithm."""
        if (self.is_ready()):
            self.fitness_values = self.evaluate()

            # run algorithm steps
            while (self.is_running()):
                for step in self.algorithm_steps:
                    # build arg dict for algorithm step and then run it

                    step_args = self.build_arg_dict_from_names(step.get_operator_args())
                    
                    self.population = step.run(**step_args)
                self. fitness_values = self.evaluate()
                # record metrics
                
                self.generation += 1
    
    def build_arg_dict_from_names(self, names):
        """Build an argument dictionary from a list of names."""
        args = {}
        for name in names:
            if (name == "self"):
                continue
            if (hasattr(self, name)):
                args[name] = getattr(self, name)
            elif (hasattr(self.problem, name)):
                args[name] = getattr(self.problem, name)
            elif (hasattr(self.stopping_condition, name)):
                args[name] = getattr(self.stopping_condition, name)
            else:
                raise AttributeError(f"Could not find attribute {name}") 

        return args

    def build_arg_dict_from_func(self, func):
        """Build an argument dictionary for a function."""
        signature = inspect.signature(func)
        # build arg dict for algorithm step
        args = {}
        for param in signature.parameters.values():
            if param.name == "self":
                continue
            args[param.name] = getattr(self, param.name)
        return args

    def is_running(self):
        """Check if the genetic algorithm should continue running."""
        stopping_condition_args = self.build_arg_dict_from_func(self.stopping_condition.is_running)
        return self.stopping_condition.is_running(**stopping_condition_args)

    def is_ready(self):
        """Check if the genetic algorithm is ready to run."""
        if (self.problem is None):
            raise ValueError("No problem has been set.")
        if (len(self.algorithm_steps) == 0):
            raise ValueError("No algorithm steps have been set.")
        if (self.stopping_condition is None):
            raise ValueError("No stopping condition has been set.")
        return True

    def evaluate(self):
        """Evaluate the population using the objective function."""
        # check if objective function exists
        if (not hasattr(self.problem, "objective_function")):
            raise Exception("The problem object should have an objective_function attribute defining the objective function to be used")
        # check if objective function is callable
        if (not callable(self.problem.objective_function)):
            raise TypeError(f"The problem objective_function should be callable. Received {type(self.problem.objective_function)}")
        # evaluate population
        return self.problem.objective_function(self.population)
    
    def get_optimum_index_and_value(self):
        """Return the optimum index and value from the population."""
        if (self.problem.goal == Optimization_Goal.MAXIMIZE):
            optimum = np.max(self.fitness_values)
            optimum_index = np.argmax(self.fitness_values, axis=0)
        elif (self.problem.goal == Optimization_Goal.MINIMIZE):
            optimum = np.min(self.fitness_values)
            optimum_index = np.argmin(self.fitness_values, axis=0)

        return optimum_index, optimum
    
    def get_worst_index_and_value(self):
        """Return the worst index and value from the population."""
        if (self.problem.goal == Optimization_Goal.MAXIMIZE):
            worst = np.min(self.fitness_values)
            worst_index = np.argmin(self.fitness_values, axis=0)
        elif (self.problem.goal == Optimization_Goal.MINIMIZE):
            worst = np.max(self.fitness_values)
            worst_index = np.argmax(self.fitness_values, axis=0)

        return worst_index, worst
    

In [ ]:
# define genetic algorithm parameters in a dictionary
ga_parameters = {
  'problem' : De_Jong_Function_5(),
  'algorithm_steps' : [Selection(selection_type="truncation"), Crossover(), Mutation()],
  'stopping_condition' : Max_Generations(1),
  'population_size' : 30,
  
}

# create a genetic algorithm object
ga = Genetic_Algorithm(**ga_parameters)

In [ ]:

ga.run()

In [ ]:
# print best vector
print(ga.population[ga.get_optimum_index_and_value()[0]])


We, Ben Vuong and Morgan Elder, certify that this report is our own, independent work and that it does not plagiarize, in part or in full, any other work.